In [14]:
import os
import random
import cProfile


from cities.queries.causal_insight import CausalInsight
from cities.utils.data_grabber import (DataGrabber, list_interventions,
                                       list_outcomes)

smoke_test = "CI" in os.environ
num_samples = 10 if smoke_test else 1000

In [15]:
interventions = list_interventions()
outcomes = list_outcomes()
shifts = [1, 2, 3]

outcome = "unemployment_rate"
intervention = "spending_commerce"
shift = 2
intervened_value = 0.7
fips = 1005

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]
values = [round(i * 0.1, 1) for i in range(1, 10)]
fips = random.choice(gdp["GeoFIPS"])

In [16]:
def basic_run():
    ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention,
        num_samples=num_samples,
    )

    ci.load_guide(forward_shift=shift)
    ci.generate_tensed_samples()
    ci.get_fips_predictions(intervened_value=intervened_value, fips=fips)
    ci.plot_predictions(range_multiplier=1)



In [17]:
profiler = cProfile.Profile()

profiler.enable()
basic_run()
profiler.disable()

# confirming that the time consuming moves involve the model
# gettign samples, etc.
profiler.print_stats(sort='cumulative')


         601839 function calls (582561 primitive calls) in 7.346 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    7.347    3.673 interactiveshell.py:3514(run_code)
        2    0.000    0.000    7.347    3.673 {built-in method builtins.exec}
        1    0.143    0.143    7.347    7.347 2408366557.py:1(<module>)
        1    0.001    0.001    7.204    7.204 2494475649.py:1(basic_run)
        1    0.000    0.000    7.013    7.013 causal_insight.py:90(generate_tensed_samples)
        3    0.008    0.003    6.975    2.325 causal_insight.py:49(generate_samples)
     12/3    0.000    0.000    6.091    2.030 module.py:1514(_wrapped_call_impl)
     12/3    0.000    0.000    6.091    2.030 module.py:1520(_call_impl)
        3    0.063    0.021    6.090    2.030 predictive.py:246(forward)
        6    0.002    0.000    6.027    1.005 predictive.py:67(_predictive)
       18    0.000    0.000    6.020  